## Soccertrack dataset - Labelbox integration
 
This is a guide for developers/annotators that contribute to the soccertrack dataset. This guide is divided into two parts. The first part is a guide for uploading data to labelbox. The second part is guide for downloading data from labelbox, converting it to the correct format and uploading it to the soccertrack dataset available on Kaggle.

### Preparation. API key and project name

Make sure to add the api key from labelbox to your environment variables, or add it to an .env that is located at the root of soccertrack. The api key can be found in the labelbox settings. The project name is the name of the project in labelbox. The project name is used to identify the correct project in labelbox.

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from dotenv import load_dotenv
import os
from soccertrack.utils import get_git_root

load_dotenv()

LABELBOX_PROJECT_ID = "cldmd6hrx08ej07yt2rydfine" # This is the project ID for the top-view dataset

LABELBOX_PROJECT_NAME = "F_Soccer_Tsukuba3" # This is the project name for the wide-view 
LABELBOX_DATASET_NAME = "Dataset_Fisheye_Soccer_Tsukuba3" # This is the dataset name for the wide-view 
LABELBOX_API_KEY = os.getenv("LABELBOX_API_KEY")

root = get_git_root()
INPUT_BBDF_DIR = root / "soccertrack"/"datasets"/"wide_view"/"annotations"

## Part1. Uploading data to labelbox

In [9]:
from pathlib import Path
import uuid
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, Dataset, Project, LabelImport, MALPredictionImport
from time import time
import requests
import soccertrack

#set up project information
client = Client(api_key=LABELBOX_API_KEY)
project = next(client.get_projects(where=Project.name == LABELBOX_PROJECT_NAME), None)
dataset = next(client.get_datasets(where=Dataset.name == LABELBOX_DATASET_NAME), None)
ontology = OntologyBuilder.from_project(project)
schema_lookup = {tool.name: tool.feature_schema_id for tool in ontology.tools}

#create a sample upload
data_row =sorted(dataset.data_rows(), key=lambda x: x.external_id)[0]
file_name = f"{data_row.external_id.split('.')[0]}.csv"
bbdf_file_path = Path(INPUT_BBDF_DIR) / file_name

#load the bbdf and convert to labelbox format
bbdf_sample = soccertrack.load_df(bbdf_file_path)

labelbox_data = bbdf_sample.to_labelbox_data(data_row, schema_lookup)

# Use MAL since LabelImport has strict API rate limits
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_job"+str(uuid.uuid4()), 
    predictions=labelbox_data)

# Wait for upload to finish
upload_job.wait_until_done() 

# Review the upload status
print(f"Done! Video_name: {data_row.external_id}, Errors: {upload_job.errors}")

## Part 2. Moving data from Labelbox to Kaggle

### 1. Downloading data from Labelbox

A script that downloads all the data from labelbox is available in `scripts/labelbox/download.py`. 

In [2]:
from labelbox import Client
import requests

client = Client(api_key=LABELBOX_API_KEY)
project = client.get_project(LABELBOX_PROJECT_ID)

export_url = project.export_labels()
exports = requests.get(export_url).json()

# View the number of videos that we have
print("Number of videos:", len(exports))

# View some specific fields of the export
print("Label ID:", exports[0]['DataRow ID'])
print("External ID:", exports[0]['External ID'])
print("Created By:", exports[0]['Created By'])
print("Created At:", exports[0]['Created At'])
print("Number of Reviews:", len(exports[0]['Reviews']))

Number of videos: 66
Label ID: cldmifpzv34ca074kator7kao
External ID: F_20200220_1_0090_0120.mp4
Created By: uchida.ikuma@image.iit.tsukuba.ac.jp
Created At: 2023-02-02T05:22:51.000Z
Number of Reviews: 0


#### Download and save videos as mp4

In [3]:
import cv2

## Download the video
video_url = exports[0]["Labeled Data"]

# Store the video url in a file
video_path = "sample_video.mp4"
with open(video_path, "wb") as file:
    file.write(requests.get(video_url).content)
    
# Get the size of the video in megabytes
video_size = os.path.getsize(video_path) / 1e6
print("Video size:", video_size, "MB")

# Get the height and width of the video
cap = cv2.VideoCapture(video_path)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Video height:", height)
print("Video width:", width)

# Get the number of frames in the video
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames:", num_frames)

Video size: 9.296215 MB
Video height: 1000
Video width: 6500
Number of frames: 750


#### Download and save annotations as csv

In [4]:
import ndjson

# Grab the annotations url
annotations_url = exports[0]["Label"]["frames"]

# Provide the appropriate authorization to view the labeled frames
headers = {"Authorization": f"Bearer {LABELBOX_API_KEY}"}
annotations = ndjson.loads(requests.get(annotations_url, headers=headers).text)

# Grab the first frame and print the contents
first_frame = annotations[0]
print("Number of objects in the first frame:", len(first_frame['objects']))

# Grab values of the first object in the first annotation
print("schemaId:", first_frame['objects'][0]['schemaId'])
print("title (object id):", first_frame['objects'][0]['title'])
print("is a keyframe?:", first_frame['objects'][0]['keyframe'])
print("bbox dimensions:", first_frame['objects'][0]['bbox'])

Number of objects in the first frame: 23
schemaId: cl39k10sy9gfd077i4dof5qls
title (object id): 0_0
is a keyframe?: True
bbox dimensions: {'top': 544, 'left': 3281, 'height': 36, 'width': 31}


### 2. Converting csv data to the correct format

In [6]:
from soccertrack.logger import show_df
from soccertrack.dataframe import BBoxDataFrame

# Lets start back at the beginning
annotations = ndjson.loads(requests.get(annotations_url, headers=headers).text)

home_team_key = '0'
away_team_key = '1'
ball_key = 'BALL'


d = {
    home_team_key: {},
    away_team_key: {},
    ball_key: {}
}

for annotation in annotations:
    for frame_annotation in annotation['objects']:
        frame_number = annotation['frameNumber']
        bbox = frame_annotation['bbox']

        if frame_annotation['title'] == ball_key:
            team_id = ball_key  
            player_id = ball_key
        else:
            team_id, player_id = frame_annotation['title'].split('_')

        if d[team_id].get(player_id) is None:
            d[team_id][player_id] = {}
        d[team_id][player_id][frame_number] = [bbox['left'], bbox['top'], bbox['width'], bbox['height']]

bbdf = BBoxDataFrame.from_dict(d)

print("bbdf.shape:", bbdf.shape)
show_df(bbdf.head())

bbdf.shape: (750, 92)


### 3. Uploading data to Kaggle

> Upload to Kaggle by hand.